# 卷积层代码实现：

### 第1件事：卷积在干什么

```
一个小窗口在图片上滑动
每到一个位置，算一个数
滑完整张图，得到一张新的（更小的）图

```

### 第2件事：会用nn.Conv2d（代码）

```python
# 创建一个卷积层
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3)
#                输入通道数=1    输出通道数=1     窗口大小3×3
```

**就像创建nn.Linear一样简单！**

```python
# 对比一下：
nn.Linear(20, 256)                    # 全连接层：20进256出
nn.Conv2d(1, 1, kernel_size=3)        # 卷积层：1通道进，1通道出，3×3的窗口
```

### 第3件事：放到网络里用

```python
net = nn.Sequential(
    nn.Conv2d(1, 1, kernel_size=3),    # 卷积层
    nn.ReLU(),                          # 激活函数
    nn.Conv2d(1, 1, kernel_size=3),    # 再来一个卷积层
)
```

**和之前学的Sequential用法一模一样！**

---

## 参数到底什么意思？

### nn.Conv2d 只有3个参数你需要关心

```python
nn.Conv2d(in_channels, out_channels, kernel_size)
```

**用快递分拣来比喻：**

```
in_channels  = 进来几条传送带（输入有几个通道）
out_channels = 出去几条传送带（输出要几个通道）  
kernel_size  = 检查员每次看多大一块区域（窗口大小）
```

### 具体例子

```python
# 黑白图片（1个通道），用3×3的窗口，输出1个通道
nn.Conv2d(1, 1, kernel_size=3)

# 彩色图片（3个通道RGB），用3×3的窗口，输出16个通道
nn.Conv2d(3, 16, kernel_size=3)

# 上一层输出16通道，这一层用5×5的窗口，输出32个通道
nn.Conv2d(16, 32, kernel_size=5)
```

**通道数就像这样理解：**

```
输入1个通道 = 黑白照片（只有亮度）
输入3个通道 = 彩色照片（红、绿、蓝）
输出16个通道 = 提取了16种不同的特征
               比如：边缘、角落、纹理、颜色变化...
               每个通道检测一种东西
```

---

## 和nn.Linear对比着记

**你已经会用nn.Linear了对吧？nn.Conv2d几乎一样！**

| | nn.Linear | nn.Conv2d |
|:---|:---|:---|
| **作用** | 处理一维数据 | 处理图片（二维数据） |
| **创建** | `nn.Linear(20, 256)` | `nn.Conv2d(1, 16, 3)` |
| **参数含义** | 输入大小, 输出大小 | 输入通道, 输出通道, 窗口大小 |
| **放进Sequential** | ✅ | ✅ 一模一样 |
| **需要手写吗** | 不需要 | 不需要 |

---

## 一个完整的对比

### 以前处理普通数据

```python
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
X = torch.rand(2, 20)          # 2个样本，20个特征
output = net(X)                 # 输出：2×10
```

### 现在处理图片数据

```python
net = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3),    # 卷积层
    nn.ReLU(),
    nn.Conv2d(16, 32, kernel_size=3),   # 再一个卷积层
)
X = torch.rand(2, 1, 28, 28)   # 2张图片，1个通道，28×28像素
output = net(X)                  # 输出：2×32×24×24
```

**唯一的区别：**

```
Linear的输入：(批量大小, 特征数)           → 2维
Conv2d的输入：(批量大小, 通道数, 高, 宽)   → 4维

为什么4维？因为图片有高和宽两个空间维度
```

---

## 边缘检测那个例子

**理解发生了什么**

```
第1步：给一张黑白条纹的图
       1 1 0 0 0 0 1 1
       白白黑黑黑黑白白

第2步：用一个[1, -1]的小窗口滑过去
       窗口看到[1,1] → 1-1=0（没变化，不是边缘）
       窗口看到[1,0] → 1-0=1（变了！是边缘！）
       窗口看到[0,0] → 0-0=0（没变化）
       窗口看到[0,1] → 0-1=-1（变了！是边缘！）

第3步：输出里非零的地方就是边缘
       0 1 0 0 0 -1 0
         ↑          ↑
       这里有边缘  这里有边缘

第4步：关键！这个[1,-1]不需要你手动设计
       PyTorch可以通过训练自己学出来！
```

---

## 你的学习路线

```
现在：
✅ 知道卷积在干什么（小窗口滑动）
✅ 会用nn.Conv2d(输入通道, 输出通道, 核大小)
✅ 知道输入是4维的(批量, 通道, 高, 宽)
✅ 能放进Sequential里用

暂时不需要：
❌ 手写corr2d
❌ 自己实现Conv2D类
❌ 手写训练循环学卷积核

以后用到再学：
⏳ padding（填充）
⏳ stride（步幅）
⏳ 池化层
```

### 记住这个模板就够了

```python
# 处理图片的网络，和之前学的用法一样
net = nn.Sequential(
    nn.Conv2d(输入通道, 输出通道, kernel_size=核大小),
    nn.ReLU(),
    nn.Conv2d(输入通道, 输出通道, kernel_size=核大小),
    nn.ReLU(),
)
```

**卷积层就是图片版的Linear层，会用nn.Linear就会用nn.Conv2d！**
---

## 三、边缘检测的例子

### 3.1 构造一个简单的输入

```python
X = torch.ones(6, 8)       # 6×8的全1矩阵
X[:, 2:6] = 0              # 中间4列设为0
print(X)
```

```
输出：
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1
1  1  0  0  0  0  1  1

左边两列是1，中间四列是0，右边两列是1
在第2列和第3列之间有一条"边缘"（从白变黑）
在第5列和第6列之间有一条"边缘"（从黑变白）
```

### 3.2 构造一个边缘检测核

```python
K = torch.tensor([[1.0, -1.0]])    # 1×2的核
```

**这个核为什么能检测边缘？**

```
核是 [1, -1]

当窗口盖住两个相同的像素时：
    [1, 1] × [1, -1] = 1×1 + 1×(-1) = 0    ← 没有边缘
    [0, 0] × [1, -1] = 0×1 + 0×(-1) = 0    ← 没有边缘

当窗口盖住两个不同的像素时：
    [1, 0] × [1, -1] = 1×1 + 0×(-1) = 1    ← 检测到边缘！
    [0, 1] × [1, -1] = 0×1 + 1×(-1) = -1   ← 检测到边缘！

相同 → 输出0（没边缘）
不同 → 输出非0（有边缘！）
```

### 3.3 做卷积

```python
Y = corr2d(X, K)
print(Y)
```

```
输出：
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0
 0   1  0  0  0 -1  0

1的那一列 = 从白变黑的边缘
-1的那一列 = 从黑变白的边缘
0 = 没有边缘
```

**卷积成功检测出了竖直方向的边缘！**

---

## 四、用梯度下降学习卷积核

### 场景

```
现在假装你不知道K是[1, -1]
你只知道输入X和输出Y
问：能不能通过训练学出K？
```

### 代码

```python
# 用PyTorch自带的Conv2d
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
```

**参数解释：**

```
nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
          ^  ^              ^^^^^        ^^^^^
          |  |              核大小       不要偏置
          |  输出通道数=1
          输入通道数=1

现在只有1个输入1个输出（黑白图片）
核大小1×2（和我们手动构造的一样）
```

---

```python
# 把X和Y变成4维（PyTorch要求的格式）
X = X.reshape((1, 1, 6, 8))    # 1个样本，1个通道，6行8列
Y = Y.reshape((1, 1, 6, 7))    # 输出也要reshape
```

**为什么要4维？**

```
PyTorch的Conv2d要求输入必须是4维：
(批量大小, 通道数, 高, 宽)

我们就1张图，1个通道
所以前面加两个1
```

---

```python
# 训练10轮
for i in range(10):
    Y_hat = conv2d(X)                    # 用当前的核算输出
    loss = (Y_hat - Y) ** 2              # 算误差
    conv2d.zero_grad()                   # 清零梯度
    loss.sum().backward()                # 反向传播
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad   # 更新权重
    if (i + 1) % 2 == 0:
        print(f'第{i+1}轮, loss={loss.sum():.3f}')
```

### 逐行翻译

```python
Y_hat = conv2d(X)
```

```
把X丢进卷积层，用当前的（随机的）核算出结果
Y_hat是预测值
```

---

```python
loss = (Y_hat - Y) ** 2
```

```
预测值 - 真实值，然后平方
平方是为了让误差都变正数
loss越小说明核越接近正确答案
```

---

```python
conv2d.zero_grad()
```

```
清零之前的梯度
每次算新梯度之前必须清零（PyTorch的规矩）
```

---

```python
loss.sum().backward()
```

```
.sum() → 把所有误差加起来变成一个数
.backward() → 反向传播，算出梯度
```

---

```python
conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
```

```
手动更新权重（梯度下降）

conv2d.weight.data  → 当前的核的值
conv2d.weight.grad  → 核的梯度（该往哪个方向调）
3e-2 = 0.03         → 学习率（每次调多少）

新权重 = 旧权重 - 学习率 × 梯度
```

### 最终结果

```python
print(conv2d.weight.data.reshape((1, 2)))
# 输出：tensor([[ 0.99, -0.99]])
```

**学出来的核接近 [1, -1]，和我们手动构造的几乎一样！**

```
我们构造的：   [1.0,  -1.0]
学出来的：     [0.99, -0.99]

非常接近！说明网络通过训练自己学会了边缘检测
```

---

## 五、总结

### 你需要记住的

```
1. 卷积运算 = 小窗口滑动，对应位置相乘再求和
2. 输出大小 = 输入大小 - 核大小 + 1
3. 卷积核是可学习的参数，不需要手动设计
4. PyTorch的Conv2d输入必须是4维：(批量, 通道, 高, 宽)
```

### 使用PyTorch自带的Conv2d

```python
# 实际中直接用这个就行，不需要自己写corr2d
conv_layer = nn.Conv2d(
    in_channels=1,        # 输入通道数
    out_channels=1,       # 输出通道数
    kernel_size=(3, 3),   # 卷积核大小
    bias=False            # 要不要偏置
)
```